# 0. Setup

Import packages and specify any important functions here.

In [1]:
# import standard python packages
import pandas as pd
import subprocess
import os
import dill

# add the utils and env directories to the path
import sys
sys.path.append('../../utils/')
sys.path.append('../../env/')

# import functions from utils directory files
from string_functions import *
from biofile_handling import *

# import paths to software installs from env
from install_locs import *

# 2. Download and describe data

This notebook collects data from the [Han et al. 2018](https://www.sciencedirect.com/science/article/pii/S0092867418301168#sec4) mouse cell atlas.

## Dataset description

- Data can be found at the [GEO Accession GSE108097](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE108097).
- This notebook previously collects data from one sample, ["Brain1"](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2906405), for the purposes of data analysis and exploration.

In [2]:
################
# general info #
################

# Specify the name of the species folder in Amazon S3
species = 'Mus_musculus'

# Specify any particular identifying conditions, eg tissue type:
conditions = 'adultbrain'

# Specify url and other variables
genome_fasta_url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.23_GRCm38.p3/GCF_000001635.23_GRCm38.p3_genomic.fna.gz'
genome_version = 'GRCm38.p3'

annot_url = 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/635/GCF_000001635.23_GRCm38.p3/GCF_000001635.23_GRCm38.p3_genomic.gff.gz'
gxc_url = 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2906nnn/GSM2906405/suppl/GSM2906405_Brain1_dge.txt.gz'

###########
# runtime #
###########

protocol = 'curl'

species_prefix = prefixify(species)

# Specify folder as destination for file downloads
output_folder = '../../output/' + species_prefix + '_' + conditions + '/'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
species_sample_dict = sample_dict(species, conditions, output_folder)


genome_fasta = url_download_biofile(
    url = genome_fasta_url,
    protocol = protocol, 
    sample_dict = species_sample_dict,
    fileclass = genome_fasta_file,
    version = genome_version
)
annot = url_download_biofile(
    url = annot_url, 
    protocol = protocol,
    sample_dict = species_sample_dict,
    fileclass = genome_gff_file,
    genome_fasta_file = genome_fasta
)
gxc = url_download_biofile(
    url = gxc_url, 
    protocol = protocol,
    sample_dict = species_sample_dict,
    fileclass = gxc_file,
    genome_fasta_file = genome_fasta
)

sample_docket = docket(species_sample_dict)
keyfiles = {
    'annot': annot,
    'genome_fasta': genome_fasta,
    'gxc': gxc
}
sample_docket.add_keyfiles(keyfiles)

display(vars(sample_docket))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  815M  100  815M    0     0  37.4M      0  0:00:21  0:00:21 --:--:-- 39.2M


NameError: name 'version' is not defined

# 3. Load in the gxc matrix and get gene names

In [3]:
genes_matrix = pd.read_csv(sample_docket.gxc.path, sep = ' ')
display(genes_matrix)

genes_list = pd.DataFrame({'gene_name':genes_matrix.index})
display(genes_list)

,Brain_1.CCGCTAAATAAATAAGGG,Brain_1.AACGCCGATCTTGCCCTC,Brain_1.ACCTGAAGTTTATCGTAA,Brain_1.CTCGCACTGAAACCGCTA,Brain_1.ATCAACATCTCTTCGGGT,Brain_1.GCGAATAGGGTCTATGTA,Brain_1.CGAGTAAGGGTCTAGTCG,Brain_1.ATCTCTTCGTAAGTTGCC,Brain_1.AACGCCTAAGGGCTCGCA,Brain_1.AACGCCTCACTTATACAG,...,Brain_1.TTGGACGCCTAGGAGATC,Brain_1.TTAACTAAAGTTTATGTA,Brain_1.GTCCCGGGACATAGGACT,Brain_1.CCGCTAGGGTTTGCTCAA,Brain_1.TGATCAGCTGTGTCAAAG,Brain_1.TCACTTGAATTATGAAGC,Brain_1.AAGTACGCTGTGTATGTA,Brain_1.CCTAGATAGAGAATTTGC,Brain_1.CATCCCATTTGCGGCTGC,Brain_1.CAAAGTGGGTTTAGCGAG
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610007P14Rik,0,3,1,2,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009B22Rik,0,3,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
0610009E02Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009L18Rik,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
n-R5s28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
n-R5s37,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
n-R5s52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
n-R5s88,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,gene_name
0,0610005C13Rik
1,0610007P14Rik
2,0610009B22Rik
3,0610009E02Rik
4,0610009L18Rik
...,...
20160,n-R5s28
20161,n-R5s37
20162,n-R5s52
20163,n-R5s88


# 4. Convert GFF genome file to GTF

In [4]:
# load in the original GFF-based annotation
models = pd.read_csv(sample_docket.annot.path, skiprows = 7, header = None, sep = '\t')
display(models)

attributes_column = 8

# Check the structure of fields in the Ensembl additional fields section
display(models[attributes_column][3])

,0,1,2,3,4,5,6,7,8
0,NC_000067.6,RefSeq,region,1.0,195471971.0,.,+,.,ID=id0;Dbxref=taxon:10090;Name=1;chromosome=1;...
1,NC_000067.6,BestRefSeq%2CGnomon,gene,3199731.0,3671742.0,.,-,.,"ID=gene0;Dbxref=GeneID:497097,MGI:MGI:3528744;..."
2,NC_000067.6,Gnomon,mRNA,3199731.0,3671742.0,.,-,.,"ID=rna0;Parent=gene0;Dbxref=GeneID:497097,Genb..."
3,NC_000067.6,Gnomon,exon,3670552.0,3671742.0,.,-,.,"ID=id1;Parent=rna0;Dbxref=GeneID:497097,Genban..."
4,NC_000067.6,Gnomon,exon,3421702.0,3421901.0,.,-,.,"ID=id2;Parent=rna0;Dbxref=GeneID:497097,Genban..."
...,...,...,...,...,...,...,...,...,...
2284598,NC_005089.1,RefSeq,region,15515.0,15558.0,.,+,.,ID=id1169987;Note=ETAS2%3B extended terminatio...
2284599,NC_005089.1,RefSeq,region,16035.0,16058.0,.,+,.,ID=id1169988;Note=CSB1%3B conserved sequencing...
2284600,NC_005089.1,RefSeq,region,16089.0,16104.0,.,+,.,ID=id1169989;Note=CSB2%3B conserved sequencing...
2284601,NC_005089.1,RefSeq,region,16114.0,16131.0,.,+,.,ID=id1169990;Note=CSB3%3B conserved sequencing...


'ID=id1;Parent=rna0;Dbxref=GeneID:497097,Genbank:XM_006495550.2,MGI:MGI:3528744;gbkey=mRNA;gene=Xkr4;product=X Kell blood group precursor related family member 4%2C transcript variant X1;transcript_id=XM_006495550.2'

In [5]:
# convert the GFF file to GTF using gffread
models_asgtf = gffread_gff_to_gtf(sample_docket.annot, species_sample_dict, gffread_loc)

In [6]:
# load the newly-generated GTF file as a dataframe
models_asgtf_df = pd.read_csv(models_asgtf.path, skiprows = 0, header = None, sep = '\t')

display(models_asgtf_df)
display(models_asgtf_df[attributes_column][1])

,0,1,2,3,4,5,6,7,8
0,NC_000067.6,Gnomon,transcript,3199731,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
1,NC_000067.6,Gnomon,exon,3199731,3207317,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
2,NC_000067.6,Gnomon,exon,3213439,3216968,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
3,NC_000067.6,Gnomon,exon,3421702,3421901,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
4,NC_000067.6,Gnomon,exon,3670552,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na..."
...,...,...,...,...,...,...,...,...,...
2224794,NC_005089.1,RefSeq,CDS,14145,15288,.,+,0,"transcript_id ""gene48832""; gene_name ""CYTB"";"
2224795,NC_005089.1,RefSeq,transcript,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""..."
2224796,NC_005089.1,RefSeq,exon,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""..."
2224797,NC_005089.1,RefSeq,transcript,15356,15422,.,-,.,"transcript_id ""rna111580""; gene_id ""gene48834""..."


'transcript_id "rna0"; gene_id "gene0"; gene_name "Xkr4";'

In [7]:
# Use a custom function to extract useful fields from the additional fields section (column 8)
# Pull from that dict to fill in additional useful columns
models_asgtf_df['field_dictionary'] = models_asgtf_df[8].apply(convert_fields_to_dict_gtf)
models_asgtf_df['gene_name'] = [d.get('gene_name') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['gene_id'] = [d.get('gene_id') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['transcript_id'] = [d.get('transcript_id') for d in models_asgtf_df['field_dictionary']]

# Remove CDS annotations because they interfere with TransDecoder cDNA generation
models_asgtf_df = models_asgtf_df[models_asgtf_df[2] != 'CDS']
display(models_asgtf_df)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,gene_id,transcript_id
0,NC_000067.6,Gnomon,transcript,3199731,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
1,NC_000067.6,Gnomon,exon,3199731,3207317,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
2,NC_000067.6,Gnomon,exon,3213439,3216968,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
3,NC_000067.6,Gnomon,exon,3421702,3421901,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
4,NC_000067.6,Gnomon,exon,3670552,3671742,.,-,.,"transcript_id ""rna0""; gene_id ""gene0""; gene_na...","{'transcript_id': 'rna0', 'gene_id': 'gene0', ...",Xkr4,gene0,rna0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224793,NC_005089.1,RefSeq,transcript,14145,15288,.,+,.,"transcript_id ""gene48832""; gene_id ""gene48832""...","{'transcript_id': 'gene48832', 'gene_id': 'gen...",CYTB,gene48832,gene48832
2224795,NC_005089.1,RefSeq,transcript,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""...","{'transcript_id': 'rna111579', 'gene_id': 'gen...",TrnT,gene48833,rna111579
2224796,NC_005089.1,RefSeq,exon,15289,15355,.,+,.,"transcript_id ""rna111579""; gene_id ""gene48833""...","{'transcript_id': 'rna111579', 'gene_id': 'gen...",TrnT,gene48833,rna111579
2224797,NC_005089.1,RefSeq,transcript,15356,15422,.,-,.,"transcript_id ""rna111580""; gene_id ""gene48834""...","{'transcript_id': 'rna111580', 'gene_id': 'gen...",TrnP,gene48834,rna111580


# 5. Generate gtf-idmm
This file maps the gene_name to `gene_id` and `transcript_id` fields generated by the conversion from GFF to GTF, which will be needed for downstream processing.

In [14]:
# Extract gene_name, gene_id, and transcript_id fields to generate an ID mapping matrix (idmm)
idmm_df = models_asgtf_df[['gene_name', 'gene_id', 'transcript_id']].drop_duplicates()
idmm_df.dropna(inplace = True)
display(idmm_df)

# generate a filename and file for the idmm
idmm_filename = '_'.join([species_prefix, conditions, 'gtf-idmm.tsv'])
idmm = idmm_file(idmm_filename, species_sample_dict, idmm_df.columns)

# save to file and add to the docket
idmm_df.to_csv(idmm.path, sep = '\t')
sample_docket.add_keyfile(idmm, 'gtf-idmm')

,gene_name,gene_id,transcript_id
0,Xkr4,gene0,rna0
8,Xkr4,gene0,rna1
15,Xkr4,gene0,rna2
22,Xkr4,gene0,rna3
29,LOC105243853,gene2,rna4
...,...,...,...
2224789,ND6,gene48830,gene48830
2224791,TrnE,gene48831,rna111578
2224793,CYTB,gene48832,gene48832
2224795,TrnT,gene48833,rna111579


# 6. Generate updated gtf
Generated an updated GTF file using transcript_id as the key. For some datasets, transcripts do not consistently get gene names and gene IDs added, which causes Transdecoder to throw errors. This resolves that problem.

In [18]:
models_asgtf_updated_df = models_asgtf_df.merge(idmm_df, on = 'transcript_id')
models_asgtf_updated_df.apply(lambda x: x['field_dictionary'].update({'gene_name': x['gene_name_y']}), axis = 1)
models_asgtf_updated_df.apply(lambda x: x['field_dictionary'].update({'gene_id': x['gene_id_y']}), axis = 1)
models_asgtf_updated_df['new_fields'] = models_asgtf_updated_df['field_dictionary'].apply(convert_dict_to_fields_gtf)
models_asgtf_updated_df = models_asgtf_updated_df[[0, 1, 2, 3, 4, 5, 6, 7, 'new_fields']]

models_asgtf_updated_filename = models_asgtf.filename.replace('.gtf', '_updated.gtf')
models_asgtf_updated = genome_gtf_file(models_asgtf_updated_filename, species_sample_dict)

models_asgtf_updated_df.to_csv(models_asgtf_updated.path, header = None, index = None, sep = '\t')

In [20]:
version = 'GRCm38p3'
format_details = 'transdecoder_cDNA'
txome_filename = '_'.join([species_prefix, conditions, version, format_details + '.fa'] )
txome_filepath = downloads_folder + txome_filename

!{transdecoder_loc + 'util/gtf_genome_to_cdna_fasta.pl'} {gtftrans_filepath} {genome_unzip_filepath} > {txome_filepath}

file_set.add(txome_filepath)
file_dict[txome_filepath] = 'organisms/' + species_folder + '/genomics_reference/transcriptome/' + txome_filename

-parsing cufflinks output: ../../output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic_transcripts.gtf
-parsing genome fasta: ../../output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.fna
^C


In [68]:
!{tdlongorf_loc} -t {txome_filepath}
!{tdpredict_loc} -t {txome_filepath}

-- Skipping CMD: /home/ec2-user/TransDecoder-TransDecoder-v5.5.0/util/compute_base_probs.pl ../../output/Drer_adultbrain/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa 0 > /home/ec2-user/2022_arcadia_glialorigins_working/notebooks/Drer_adultbrain/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder_dir/base_freqs.dat, checkpoint [/home/ec2-user/2022_arcadia_glialorigins_working/notebooks/Drer_adultbrain/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder_dir.__checkpoints_longorfs/base_freqs_file.ok] exists.
-skipping long orf extraction, already completed earlier as per checkpoint: /home/ec2-user/2022_arcadia_glialorigins_working/notebooks/Drer_adultbrain/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder_dir.__checkpoints_longorfs/TD.longorfs.ok
-- Skipping CMD: /home/ec2-user/TransDecoder-TransDecoder-v5.5.0/util/get_top_longest_fasta_entries.pl Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder_dir/longest_orfs.cds 5000 5000 > Drer_adultbrain_GRCz10_transdecoder_c

# 13. Push files to AWS S3

Iteratively moves through the file_set and file_dict variables and populates files into the right place in AWS.

In [5]:
for filename in file_set:
    input = filename
    path = 's3://arcadia-reference-datasets/' + file_dict[filename]
    !aws s3 cp $input $path

upload: ../../output/Mmus_adultbrain/GSM2906406_Brain2_dge.txt to s3://arcadia-reference-datasets/organisms/Mus_musculus/functional_sequencing/scRNA-Seq/GSM2906406_Brain2_dge.txt
upload: ../../output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.gff to s3://arcadia-reference-datasets/organisms/Mus_musculus/genomics_reference/annotation/GCF_000001635.23_GRCm38.p3_genomic.gff
upload: ../../output/Mmus_adultbrain/GCF_000001635.23_GRCm38.p3_genomic.fna to s3://arcadia-reference-datasets/organisms/Mus_musculus/genomics_reference/genome/GCF_000001635.23_GRCm38.p3_genomic.fna


In [69]:
suffixes = ['.bed', '.cds', '.gff3', '.pep']
filenames_forexport = [txome_filename + '.transdecoder' + i for i in suffixes]
display(filenames_forexport)

for filepath in filenames_forexport:
    s3address = 's3://arcadia-reference-datasets/organisms/' + species_folder + '/genomics_reference/proteome/' + filepath
    !aws s3 cp {'./' + filepath} {s3address}

['Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.bed',
 'Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.cds',
 'Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.gff3',
 'Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.pep']

upload: ./Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.bed to s3://arcadia-reference-datasets/organisms/Danio_rerio/genomics_reference/proteome/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.bed
upload: ./Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.cds to s3://arcadia-reference-datasets/organisms/Danio_rerio/genomics_reference/proteome/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.cds
upload: ./Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.gff3 to s3://arcadia-reference-datasets/organisms/Danio_rerio/genomics_reference/proteome/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.gff3
upload: ./Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.pep to s3://arcadia-reference-datasets/organisms/Danio_rerio/genomics_reference/proteome/Drer_adultbrain_GRCz10_transdecoder_cDNA.fa.transdecoder.pep
